In [5]:
import torch
import torch.nn as nn
# from DenseNetV2 import DenseNet
x = torch.randn(1, 3, 224, 224)
# model = DenseNet()
# print(model)
# out = model.forward(x)
# out.shape

In [20]:
class DenseLayer(nn.Module):
    def __init__(self, in_feature, growth_rate):
        super().__init__()
        inter_channel = 4*growth_rate
        self.bn1 = nn.BatchNorm2d(in_feature)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_channels=in_feature, out_channels=inter_channel, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(inter_channel)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(in_channels=inter_channel, out_channels=growth_rate, kernel_size=3, stride=1, padding=1, bias=False)
    
    def forward(self, x):
        out = self.conv1(self.relu1(self.bn1(x)))
        out = self.conv2(self.relu2(self.bn2(out)))
        return torch.cat([x, out], 1)

In [21]:
denselayer = DenseLayer(64, 32)
a

DenseLayer(
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU(inplace=True)
  (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU(inplace=True)
  (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
)

In [45]:
denselayer.items()

AttributeError: 'DenseLayer' object has no attribute 'items'

In [22]:
x = torch.randn(1, 64, 56, 56)
out = denselayer.forward(x)
out.shape

torch.Size([1, 96, 56, 56])

In [83]:

class DenseBlock(nn.Module):
    def __init__(self, in_feature, num_layer, growth_rate):
        super().__init__()
        for idx in range(num_layer):
            input_feature = in_feature + idx * growth_rate
            layer = DenseLayer(input_feature, growth_rate)
            self.add_module("denselayer%d"%(idx+1), layer)
    def forward(self, init_feature):
        features = [init_feature]
        for name, layer in self.named_children():
            new_features = layer(features[-1])
            features.append(new_features)
        return torch.cat(features, 1)

In [84]:
block = DenseBlock(64, 2, 32)
block(x).shape
# block

torch.Size([1, 288, 56, 56])

In [67]:
for name, layer in block.named_children():
    print(name)
    print(layer)
    layer(x)
    break

denselayer1
DenseLayer(
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU(inplace=True)
  (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU(inplace=True)
  (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
)


In [77]:
features = [x]
new_features = layer(x)

In [78]:
new_features.shape

torch.Size([1, 96, 56, 56])

In [79]:
features.append(new_features)
torch.cat(features, 1).shape

torch.Size([1, 160, 56, 56])